<a href="https://colab.research.google.com/github/Joon2000/machinelearning/blob/main/5_2_%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
"""교차 검증과 그리드 서치"""

# 검증세트(validation set) 훈련세트를 검증세트와 테스트세트로 나눈다 (보통 20~30%로 만)
# 가장 좋은 모델을 고르고 훈련세트와 검증세트를 합쳐서 다시 

import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target,test_target = train_test_split(data, target, test_size=0.2, random_state=42) #훈련 데이터와 테스트 데이터 나누기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42) #훈련 세트와 검증 세트로 나누기
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
#과데접합

0.9971133028626413
0.864423076923077


In [7]:
#교차 검증(cross validiation) 검증 세트를 만드는 과정을 여러번 반복함
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)  #5-폴드 교차 검증
print(scores)

{'fit_time': array([0.01191545, 0.0071857 , 0.00762892, 0.00742197, 0.00746012]), 'score_time': array([0.00124693, 0.00111198, 0.00113106, 0.00133681, 0.00107074]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
import numpy as np
print(np.mean(scores['test_score'])) #평균

0.855300214703487


In [11]:
#교차 검증을 할 때 훈련 세트를 섞으려면 분할기(splitter)를 지정해야 함
#위의 과저에서는 train_test_split()이 섞어줌
#회귀 모델은 KFold 분할기, 분류 모델의 경우 stratifiedKFold를 사용

from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold()) #cv를 알아서 인식해주기는 함
print(np.mean(scores['test_score']))

0.855300214703487


In [13]:
# 10-폴드 교차 검증
splitter = StratifiedKFold(n_splits = 10, shuffle=True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [14]:
#하이퍼파라미터 튜닝
#한 매개변수의 최적값은 다른 매개변수의 값이 바뀌면 함께 달라짐
#해결법 => 그리드서치(Grid Search)
#사이킷런의 GridSearchCV는 하이퍼파라미터 탐색과 교차 검증을 한번에 수행

#min_impurity_decrease 매개변수의 최적값을 찾아보자
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42),params, n_jobs=-1)

#GridSearchCV의 기본 cv값은 5이다 => 따라서 5*5=25개의 모델을 훈련한다
#n_jobs는 병렬 실행에 사용할 CPU 코어의 수 (-1은 모두 사용)

gs.fit(train_input, train_target)
#그리드 서치는 마지막에 전체 훈련 세트로 다시 모델을 훈련해줌

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [15]:
print(gs.best_params_) #최적의 파라미터

{'min_impurity_decrease': 0.0001}


In [16]:
#교차 검증의 평균 점수
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [17]:
#np.argmax() 가장 큰 값의 인덱스 추출
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [19]:
#params 수 늘리기
params = {'min_impurity_decrease': np.arange(0.0001,0.001, 0.0001), 'max_depth': range(5,20,1), 'min_samples_split': range(2,100,10)}
#총 (9*15*10)*5=6750 모델을 만듬

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [20]:
print(np.max(gs.cv_results_['mean_test_score']))
#0.0001이나 1같은 간격을 둔 것에 특별한 의미가 없다

0.8683865773302731


In [21]:
#랜덤 서치
#범위나 간격을 미리 정하기 어려울 때, 너무 많은 매개변수 조건이 있을 때

from scipy.stats import uniform, randint
rgen = randint(0,10)
rgen.rvs(10)

array([5, 8, 0, 2, 0, 2, 0, 6, 0, 3])

In [22]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 90,  94,  88, 106,  95, 108, 111, 107,  95, 106]))

In [23]:
ugen= uniform(0,1)
ugen.rvs(10)

array([0.88114808, 0.30402987, 0.96520045, 0.9170597 , 0.93864045,
       0.19717193, 0.12781566, 0.35556281, 0.21663714, 0.7752668 ])

In [25]:
params = {"min_impurity_decrease": uniform(0.0001, 0.001), 'max_depth': randint(20,50), 'min_samples_split': randint(2,25), 'min_samples_leaf': randint(1, 25)}

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42) #총 n_iter만큼 샘플링
gs.fit(train_input, train_target)
#그리드서치보다 교차검증 수를 줄이고 더 넓은 영역을 효과적으로 탐색
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [26]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [27]:
dt = gs.best_estimator_ #최적의 모델은 이미 전체 훈련 세트(train_input, train_target)로 훈련되어 best_estimator_에 저장되어 있음
print(dt.score(test_input, test_target))

0.86


In [ ]:
#교차검증으로 매개변수(하이퍼파라미터)를 정하고 => 훈련 데이터로 훈련